# Домашнее задание: класс линейной регрессии

## Импорт библиотек, установка констант

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [84]:
RANDOM_STATE = 42
TEST_SIZE = 0.25
np.random.seed(42)

## Интерфейс Scikit-Learn

Scikit-Learn (`sklearn`)- библиотека, в которой реализованы практически все используемые сегодня алгоритмы машинного обучения.

Для реализации алгоритмов машинного обучения в `sklearn` всегда используется один интерфейс - класс с функциями `fit(X, y)` для обучения модели по обучающей выборке `X`, `y` и `predict(X)` для возвращения предсказаний на выборке `X`. При создании класса можно указывать дополнительные параметры, влияющие на работу алгоритма машинного обучения.

## Линейная регрессия

from sklearn.linear_model import LinearRegressionРеализуйте класс линейной регрессии c L2-регуляризацией со следующей логикой:
*   При создании класса задайте коэффициент регуляризации `reg_coef`, равный по умолчанию нулю, а также веса (=None)
*   Задача функции `fit` - по выборке `X` и `y` найти веса `w` и сохранить их внутри класса в `self.w`:  
$w = (X^TX + \lambda I)^{-1}X^Ty,$
где $\lambda$ - коэффициент регуляризации, $I$ - единичная матрица.
  
P.S. Формула верна только при наличии вектора признаков, равного 1 - поэтому для вашего удобства мы уже добавили его в класс.

*   Задача функции `predict` - по весам `self.w` и `X` вернуть предсказания  


In [102]:
from sklearn.base import BaseEstimator, RegressorMixin

class LinearRegressors(BaseEstimator, RegressorMixin):

    def __init__(self, reg_coef=0.0):
        # Коэффициент регуляризации
        self.reg_coef = reg_coef

        # Веса модели
        self.w = None


    def fit(self, X, y):
        # Добавляем вектор признаков, равных 1
        X = np.hstack([np.ones((len(X), 1)), X])

        # Формируем матрицы для расчета весов
        XtX = X.T.dot(X)
        lambda_I = self.reg_coef * np.identity(XtX.shape[0])
        XtY = X.T.dot(y)

        # Рассчитываем веса
        self.w = np.linalg.solve(XtX + lambda_I, XtY)

        return self

    def predict(self, X):
        # Проверка входных данных
        # Добавляем вектор признаков, равных 1
        X = np.hstack([np.ones((len(X), 1)), X])

        # Прогнозируем значения
        y_pred = X.dot(self.w)

        return y_pred
    @property
    def coef_(self):
        """
        Возвращает коэффициенты модели (весовые коэффициенты).
        """
        return self.w[1:]  # Исключаем первый коэффициент, связанный с константой

    @property
    def intercept_(self):
        """
        Возвращает свободный член (константу) модели.
        """
        return self.w[0]

Если бы не использовали класс, нам пришлось бы передавать веса `w` в функцию `predict()` каждый раз, когда мы захотели бы сделать предсказания, а так они хранятся внутри класса. Это особенно удобно, если таких вспомогательных переменных много.

Будем тестировать ваш класс на датасете о стоимости домов в Калифорнии.

In [94]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

Разобъем данные на тренировочную и тестовую часть.

In [95]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
    )

Масштабируем данные.

In [96]:
sc = StandardScaler()

sc.fit(X_train)

X_train = pd.DataFrame(sc.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

## Задание 1

Обучите вашу линейную регрессию (без регуляризации) на тренировочных данных. Выведите на экран максимальный по модулю вес (со знаком). Ответ округлите до сотых.

In [103]:
# ваш код здесь
model = LinearRegressors()

#model.fit(X_train, y_train)

In [104]:
#pd.DataFrame(model.coef_, X_train.columns, columns=["coef"]).sort_values(
#    by="coef", ascending=False)
model.fit(X_train, y_train)

LinearRegressors()

In [105]:
weights = model.coef_
bias = model.intercept_

# Объединяем все параметры модели (веса и смещение)
all_params = np.concatenate((weights, [bias]))

# Находим максимальный по модулю вес
max_param = np.max(np.abs(all_params))

# Округляем до двух знаков после запятой
rounded_max_param = round(max_param, 2)

print(f'Максимальный по модулю вес: {rounded_max_param}')


Максимальный по модулю вес: 2.07


## Задание 2

Сделайте прогноз на тестовых данных и выведите на экран значение метрики $R^2$ на тесте.  
Ответ округлите до тысячных.

In [106]:
# ваш код здесь
# Прогнозирование на тестовом наборе
y_pred = model.predict(X_test)

# Вычисление метрики R^2 на тесте
r2_score_value = model.score(X_test, y_test)
print(f"R^2 on test set: {r2_score_value}")

R^2 on test set: 0.5910509795491351


## Задание 3

Теперь обучите линейную регрессию с коэффициентом регуляризации $\alpha = 100$ на тренировочных данных.

Чему теперь равен наибольший по модулю вес? Ответ округлите до сотых.

In [107]:
# ваш код здесь
model = LinearRegressors(100)

model.fit(X_train, y_train)

LinearRegressors(reg_coef=100)

In [108]:
weights = model.coef_
bias = model.intercept_

# Объединяем все параметры модели (веса и смещение)
all_params = np.concatenate((weights, [bias]))

# Находим максимальный по модулю вес
max_param = np.max(np.abs(all_params))

# Округляем до двух знаков после запятой
rounded_max_param = round(max_param, 2)

print(f'Максимальный по модулю вес: {rounded_max_param}')


Максимальный по модулю вес: 2.06


## Задание 4

Для модели с регуляризацией сделайте прогноз на тестовых данных и выведите на экран значение метрики $R^2$ на тесте.  
Ответ округлите до тысячных.

In [109]:
# ваш код здесь
# ваш код здесь
# Прогнозирование на тестовом наборе
y_pred = model.predict(X_test)

# Вычисление метрики R^2 на тесте
r2_score_value = model.score(X_test, y_test)
print(f"R^2 on test set: {r2_score_value}")

R^2 on test set: 0.5924168923824344
